# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff724a5ff70>
├── 'a' --> tensor([[ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840]])
└── 'x' --> <FastTreeValue 0x7ff724a565e0>
    └── 'c' --> tensor([[ 0.1305,  0.4034,  0.2404, -0.5426],
                        [-0.0571, -0.3861, -0.0041,  0.3200],
                        [-0.3360,  0.1568, -2.0318, -0.7948]])

In [4]:
t.a

tensor([[ 0.4193, -1.4295, -1.6368],
        [-0.0607, -1.4455,  1.0840]])

In [5]:
%timeit t.a

60.7 ns ± 0.0545 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff724a5ff70>
├── 'a' --> tensor([[-0.9023, -0.2208, -0.9135],
│                   [-0.3319, -0.1236, -0.5392]])
└── 'x' --> <FastTreeValue 0x7ff724a565e0>
    └── 'c' --> tensor([[ 0.1305,  0.4034,  0.2404, -0.5426],
                        [-0.0571, -0.3861, -0.0041,  0.3200],
                        [-0.3360,  0.1568, -2.0318, -0.7948]])

In [7]:
%timeit t.a = new_value

63.9 ns ± 0.0178 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4193, -1.4295, -1.6368],
               [-0.0607, -1.4455,  1.0840]]),
    x: Batch(
           c: tensor([[ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4193, -1.4295, -1.6368],
        [-0.0607, -1.4455,  1.0840]])

In [11]:
%timeit b.a

55 ns ± 0.00742 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8551, -1.5504,  0.2998],
               [ 0.4421,  0.1085,  0.8342]]),
    x: Batch(
           c: tensor([[ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.0952 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 0.378 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 37.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 798 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff724acb0a0>
├── 'a' --> tensor([[[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]],
│           
│                   [[ 0.4193, -1.4295, -1.6368],
│                    [-0.0607, -1.4455,  1.0840]]])
└── 'x' --> <FastTreeValue 0x7ff724ab8e50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 74.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff66fce98e0>
├── 'a' --> tensor([[ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840],
│                   [ 0.4193, -1.4295, -1.6368],
│                   [-0.0607, -1.4455,  1.0840]])
└── 'x' --> <FastTreeValue 0x7ff66fce9880>
    └── 'c' --> tensor([[ 0.1305,  0.4034,  0.2404, -0.5426],
                        [-0.0571, -0.3861, -0.0041,  0.3200],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 70 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.6 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.1305,  0.4034,  0.2404, -0.5426],
                       [-0.0571, -0.3861, -0.0041,  0.3200],
                       [-0.3360,  0.1568, -2.0318, -0.7948]],
              
                      [[ 0.1305,  0.4034,  0.2404, -0.5426],
                       [-0.0571, -0.3861, -0.0041,  0.3200],
                       [-0.3360,  0.1568, -2.0318, -0.7948]],
              
                      [[ 0.1305,  0.4034,  0.2404, -0.5426],
                       [-0.0571, -0.3861, -0.0041,  0.3200],
                       [-0.3360,  0.1568, -2.0318, -0.7948]],
              
                      [[ 0.1305,  0.4034,  0.2404, -0.5426],
                       [-0.0571, -0.3861, -0.0041,  0.3200],
                       [-0.3360,  0.1568, -2.0318, -0.7948]],
              
                      [[ 0.1305,  0.4034,  0.2404, -0.5426],
                       [-0.0571, -0.3861, -0.0041,  0.3200],
                       [-0.3360,  0.1568, -2.0318, -0.7948]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948],
                      [ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948],
                      [ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948],
                      [ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948],
                      [ 0.1305,  0.4034,  0.2404, -0.5426],
                      [-0.0571, -0.3861, -0.0041,  0.3200],
                      [-0.3360,  0.1568, -2.0318, -0.7948],
                      [ 0.1305,  0.4034,  0.2404, -0.5426],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

302 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
